In [2]:
DIRECTORY = "C:\\Work\\teamwork-visualiser-dashboard\\server\\saved_data"

import pandas as pd

In [6]:
# Load the data

PATH = DIRECTORY + "\\291\\" + 'heart rate-red.csv'
df = pd.read_csv(PATH, parse_dates=['Server Time'], index_col='Server Time')

# Resample the data to 1Hz frequency, taking the last value if there are multiple within one second
resampled_df = df.resample('1S').last()

# Drop rows with NaN values that may appear after resampling
resampled_df.fillna(method='ffill', inplace=True)

# Save the resampled data to a new CSV file
resampled_df.to_csv('resampled_data.csv')

C:\Users\riord\AppData\Local\Temp\ipykernel_5760\1033293908.py:7: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  resampled_df = df.resample('1S').last()
C:\Users\riord\AppData\Local\Temp\ipykernel_5760\1033293908.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  resampled_df.fillna(method='ffill', inplace=True)


In [ ]:
# Load the positioning data
position_df = pd.read_csv('position.csv')

# Load the heart rate data
heart_rate_df = pd.read_csv('heart_rate.csv')

# Convert 'audio time' to a timedelta and add it to 'audio_start_timestamp' to align timestamps
position_df['audio_start_timestamp'] = pd.to_datetime(position_df['audio_start_timestamp'], unit='s')
position_df['audio_time_td'] = pd.to_timedelta(position_df['audio time'])
position_df['aligned_timestamp'] = position_df['audio_start_timestamp'] + position_df['audio_time_td']

# Ensure the heart rate timestamp is in datetime format
heart_rate_df['timestamp'] = pd.to_datetime(heart_rate_df['timestamp'])

# Merge the dataframes on the nearest 'aligned_timestamp'
merged_df = pd.merge_asof(position_df.sort_values('aligned_timestamp'),
                          heart_rate_df[['timestamp', 'heart_rate']].sort_values('timestamp'),
                          left_on='aligned_timestamp', right_on='timestamp',
                          direction='nearest')

# Drop the extra columns and save the merged dataframe to a new CSV file
merged_df.drop(columns=['audio_start_timestamp', 'audio_time_td', 'timestamp'], inplace=True)
merged_df.to_csv('merged_position_heart_rate.csv', index=False)